In [1]:
import pandas as pd
from biblib import Entry
import pybtex as pbt
import math
import numpy as np
import re

## Load data needed to generate tables

In [3]:
path = '~/Downloads/Simple Decision Rules Give High Accuracy for Detecting Social Bots on Benchmark Datasets - Bot detection papers(18).tsv'
#path = 'metrics.tsv'

df = pd.read_csv(path, sep='\t')
df.fillna("", inplace=True)

In [31]:
dataset_df_path = '~/Downloads/Simple Decision Rules Give High Accuracy for Detecting Social Bots on Benchmark Datasets - datasets(15).tsv'

dataset_df = pd.read_csv(dataset_df_path, sep='\t')
dataset_df.fillna("", inplace=True)


In [5]:
scores_path = '~/work/repo/bot-detection/scores.csv'
sdt_df = pd.read_csv(scores_path)


## Generate bibliography methods.bib

In [6]:
bib = pbt.database.BibliographyData()

def add_bib_entries(df):
    for row in df.to_dict(orient="records"):
        #print(row)
        if row['bibtex_id'] in bib.entries.keys():
            continue
        if 'analyzed?' in row:
            if not row['analyzed?']:
                continue
        if not row['bibtex_id']:
            continue
        inputdict = {
            'author': row['authors'],
            'title': row['title'],
            'year': str(int(row['year']))
        }

        if row['conference?']:
            inputdict.update({
                'booktitle': row['booktitle'],
                'pages': row['pages'],
            })
            if row['booktitle']:
                inputdict['booktitle'] = row['booktitle'] 
            type_ = 'inproceedings'
        else:
            inputdict['journal'] = row['journal']
            if row['volume']:
                inputdict['volume'] = str(int(row['volume']))
            if row['number']:
                inputdict['number'] = str(int(row['number']))
            type_ = 'article'
        if row['publisher']:
            inputdict['publisher'] = row['publisher']
        if row['doi']:
            inputdict['doi'] = row['doi']
        if row['pages']:
            inputdict['pages'] = row['pages']
        entry = pbt.database.Entry(type_=type_, fields=inputdict)
        bib.add_entry(entry=entry, key=row['bibtex_id'])

add_bib_entries(df)
add_bib_entries(dataset_df)

bib.to_file("methods.bib")

## Generate table dataset -> paper that uses it

In [7]:
dataset_dict = {}

for row in df.to_dict(orient="records"):
    datasets = row['dataset(s) used'].split("; ")
    for d in datasets:
        if d in dataset_dict:
            dataset_dict[d].append(row['bibtex_id'])
        else:
            dataset_dict[d] = [row['bibtex_id']]

In [8]:
for k,v in dataset_dict.items():
    impl_papers = ", ".join(dataset_dict[k])

    cite_as = '\\cite{' + impl_papers + '} \\\\'
    dataset_name = k
    print(dataset_name+ " & " + cite_as)

twibot-2020 & \cite{feng2022heterogeneity-aware, , , feng2021botrgcn, geng2021satar, dehghan2018detecting} \\
cresci-rtbust-2019 & \cite{guo2022social, , yang2020scalable, sayyadiharikandeh2020detection, , , , mazza2019rtbust} \\
botometer-feedback-2019 & \cite{guo2022social, , yang2020scalable, sayyadiharikandeh2020detection} \\
gilani-2017 & \cite{guo2022social, dimitriadis2021social, , yang2020scalable, gilani2020classification, sayyadiharikandeh2020detection, , echeverria2018lobo} \\
cresci-stock-2018 & \cite{guo2022social, dimitriadis2021social, , , yang2020scalable, sayyadiharikandeh2020detection} \\
midterm-2018 & \cite{guo2022social, dimitriadis2021social, , yang2020scalable, sayyadiharikandeh2020detection, } \\
cresci-2015 & \cite{gonzalez2022the, , dimitriadis2021social, , stella2019influence, echeverria2018lobo, cresci2015fame} \\
cresci-2017 & \cite{gonzalez2022the, thavasimani2022a, heidari2021an, ilias2021detecting, geng2021satar, dimitriadis2021social, , yang2020scalable

## Generate table for dataset, #people/bots, description

In [34]:
from functools import reduce

benchmark_only = False

for row in dataset_df.to_dict(orient="records"):
    if not row['analyzed?']:
        continue
    if benchmark_only and row['benchmark?'] != '1':
        continue
    if not benchmark_only and row['benchmark?'] == '1':
        continue
    name = "\\data{" + row['dataset name'] + "}"
        
    num_humans = int(row['# humans we have']) if row['# humans we have']!="" else '-'
    num_bots = int(row['# bots we have']) if row['# bots we have']!="" else '-'
    
    if benchmark_only:
        desc_list = row['description'].split("; ")
        if len(desc_list) == 1:
            desc = row['description']
        else:
            desc_list = ['\\data{' + d + '}' for d in desc_list]
            desc = ", ".join(desc_list)
    else:
        desc = row['description']
    print(
        name + \
        " & \\cite{" + \
        row['bibtex_id'] + \
        "} " + \
        f"& {num_humans} & {num_bots} & {desc}\\\\"
    )

\data{botwiki-2019} & \cite{yang2020scalable} & 0 & 698 & Labeled datasets available in the literature and three new ones.\\
\data{celebrity-2019} & \cite{yang2019arming} & 5918 & 0 & Celebrity accounts.\\
\data{the-fake-project-2015} & \cite{cresci2015fame} & 469 & 0 & Followers of @TheFakeProject who completed unique CAPTCHA.\\
\data{elezioni-2015} & \cite{cresci2015fame} & 1488 & 0 & Users of \#elezioni2013, excluding political actors and media; manually verified.\\
\data{fake-followers-2015} & \cite{cresci2017the} & 0 & 3351 & Accounts that inflate another account's number of followers.\\
\data{genuine-accounts-cresci} & \cite{cresci2017the} & 3474 & 0 & Random\footnote{We describe in \Cref{sec:background} our reason for believing this sample is a convenience sample from previous work.} sample of verified, human-operated accounts through hybrid crowdsensing.\\
\data{social-spambots-1} & \cite{cresci2017the} & 0 & 991 & Retweeters of an Italian political candidate during 2014 campai

## Generate table for dataset -> sdt/sota scores

In [6]:
def get_score(score, dataset_name):
    if score == "":
        return -1
    match = re.search(f"all: ([0-9]*(\.\d+)?)", score)
    if match:
        return -1
    match = re.search(f"{dataset_name}: ([0-9]*(\.\d+)?)", score)
    if match: 
        return float(match.group(1))
    return float(score)

In [7]:
def get_max_score(df, dataset_name, metric):
    scores = df[df['dataset(s) used'].str.contains(dataset_name)][metric].map(lambda x: get_score(x, dataset_name))
    max_score_ind = scores.idxmax()
    return scores.loc[max_score_ind], df.at[max_score_ind, 'bibtex_id']


In [8]:
df[df['dataset(s) used'].str.contains('yang-2013')]['f1'].map(lambda x: get_score(x, 'yang-2013'))


34   -1.0
67    0.9
Name: f1, dtype: float64

In [18]:
dataset_names = [
    
    'caverlee-2011',
    'cresci-2015',
    'cresci-2017',
    #'yang-2013',
    'botometer-feedback-2019',
    'midterm-2018',
    'cresci-rtbust-2019',
    'pan-2019',
    'gilani-2017',
    #'cresci-stock-2018',
    'twibot-2020',
]

score_dict = {}

for name in dataset_names:
    score_dict[name] = {
        'accuracy': get_max_score(df, name, 'accuracy'),
        'f1': get_max_score(df, name, 'f1'),
    }

In [19]:
def print_single_dataset_score_table(score_dict):
    max_depth = 5
    tolerance = 0.025

    for k,v in score_dict.items():
        accuracy_sota = float(v['accuracy'][0])
        f1_sota = float(v['f1'][0])
        row = sdt_df[sdt_df['name'] == k].to_dict(orient="records")[0]
        accuracies = [row[f'a{i}'] for i in range(1, max_depth+1)]
        a_max_ind = np.argmax(accuracies)
        f1s = [row[f'f{i}'] for i in range(1, max_depth+1)]
        f_max_ind = np.argmax(f1s)
        accuracy_sdt = accuracies[a_max_ind]
        f1_sdt = f1s[f_max_ind]

        for i, acc in enumerate(accuracies):
            if accuracy_sdt - acc <= tolerance:
                a_max_ind = i
                accuracy_sdt = acc
                break
        for i, f in enumerate(f1s):
            if f1_sdt - f <= tolerance:
                f_max_ind = i
                f1_sdt = f
                break



        accuracy_diff = accuracy_sdt - accuracy_sota
        f1_diff = f1_sdt - f1_sota
        if v['accuracy'][1] == v['f1'][1]:
            cite = f"{v['accuracy'][1]}"
        else:
            cite = f"{v['accuracy'][1]}, {v['f1'][1]}"
        sepa = '\\phantom{-}' if accuracy_diff > 0 else ''
        sepf = '\\phantom{-}' if f1_diff > 0 else ''
        print('\\data{' + f"{k}" + "} & " + f"{accuracy_sdt:0.2f}" + f"/{f1_sdt:0.2f} " \
    #           + " \\textit{" \
    #           + f"({a_max_ind+1})" \
    #           + "} & " \
              + f"& {f_max_ind+1}" \
              + " & " \
    #           + f"{accuracy_sota:0.2f}" \
    #           + " \\cite{" + f"{v['accuracy'][1]}" \
    #           + "} & " \
    #           + f"{f1_sota:0.2f}" \
              + " \\cite{" \
              + cite + "} & " \
              + f"{sepa}{accuracy_diff:0.2f}/{sepf}{f1_diff:0.2f} \\\\")

\data{caverlee-2011} & 0.93/0.91 & 2 &  \cite{lee2011a} & -0.06/-0.07 \\
\data{cresci-2015} & 0.98/0.98 & 3 &  \cite{cresci2015fame} & -0.01/-0.01 \\
\data{cresci-2017} & 0.98/0.98 & 1 &  \cite{kudugunta2018deep} & -0.02/-0.02 \\
\data{botometer-feedback-2019} & 0.79/0.56 & 4 &  \cite{guo2022social} & -0.02/-0.14 \\
\data{midterm-2018} & 0.97/0.98 & 1 &  \cite{giorgi2021a} & -0.01/\phantom{-}0.01 \\
\data{cresci-rtbust-2019} & 0.69/0.73 & 1 &  \cite{mazza2019rtbust} & -0.24/-0.14 \\
\data{pan-2019} & 0.93/0.93 & 3 &  \cite{geng2021satar} & -0.02/-0.03 \\
\data{gilani-2017} & 0.81/0.80 & 2 &  \cite{gilani2020classification} & -0.05/-0.03 \\
\data{twibot-2020} & 0.82/0.86 & 1 &  \cite{feng2022heterogeneity-aware} & -0.05/-0.03 \\


In [ ]:
for k,v in leave_one_out_dict.items():
    accuracy_sota = float(v['accuracy'][0])
    f1_sota = float(v['f1'][0])
    row = sdt_df[sdt_df['name'] == k].to_dict(orient="records")[0]
    accuracies = [row[f'a{i}'] for i in range(1, max_depth+1)]
    a_max_ind = np.argmax(accuracies)
    f1s = [row[f'f{i}'] for i in range(1, max_depth+1)]
    f_max_ind = np.argmax(f1s)
    accuracy_sdt = accuracies[a_max_ind]
    f1_sdt = f1s[f_max_ind]
    
    for i, acc in enumerate(accuracies):
        if accuracy_sdt - acc <= tolerance:
            a_max_ind = i
            accuracy_sdt = acc
            break
    for i, f in enumerate(f1s):
        if f1_sdt - f <= tolerance:
            f_max_ind = i
            f1_sdt = f
            break
    
    
    
    accuracy_diff = accuracy_sdt - accuracy_sota
    f1_diff = f1_sdt - f1_sota
    if v['accuracy'][1] == v['f1'][1]:
        cite = f"{v['accuracy'][1]}"
    else:
        cite = f"{v['accuracy'][1]}, {v['f1'][1]}"
    sepa = '\\phantom{-}' if accuracy_diff > 0 else ''
    sepf = '\\phantom{-}' if f1_diff > 0 else ''
    print('\\data{' + f"{k}" + "} & " + f"{accuracy_sdt:0.2f}" + f"/{f1_sdt:0.2f} " \
#           + " \\textit{" \
#           + f"({a_max_ind+1})" \
#           + "} & " \
          + f"& {f_max_ind+1}" \
          + " & " \
#           + f"{accuracy_sota:0.2f}" \
#           + " \\cite{" + f"{v['accuracy'][1]}" \
#           + "} & " \
#           + f"{f1_sota:0.2f}" \
          + " \\cite{" \
          + cite + "} & " \
          + f"{sepa}{accuracy_diff:0.2f}/{sepf}{f1_diff:0.2f} \\\\")